---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [79]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    # read university_towns.txt
    towns = pd.read_csv('university_towns.txt',sep='\n', header=None, names=['RegionName'])

    # State = contains [edit]. Otherwise = NaN
    towns['State'] = np.where(towns['RegionName'].str.contains('\[edit\]'),towns['RegionName'],np.NaN)
    
    # Forward fill state names in place of NaN
    towns['State'].fillna(method='ffill',inplace=True)
    
    # Reorder columns
    towns = towns[['State','RegionName']]
    
    # Remove rows where State and RegionName are the same
    towns = towns[towns['State'] != towns['RegionName']]  
    
    towns['State'] = towns.apply(lambda x: x['State'][:x['State'].find("[")], axis = 1)
    towns['RegionName'] = towns.apply(lambda x: x['RegionName'][:x['RegionName'].find("(")] if x['RegionName'].find("(") > -1 else x['RegionName'], axis = 1)
    towns['State'] = towns['State'].str.strip()
    towns['RegionName'] = towns['RegionName'].str.strip()

    towns = towns.reset_index(drop=True)
    
    # Using pseudocode from notes
#     data = []
#     state = None
#     state_towns = []
#     with open('university_towns.txt') as file:
#         for line in file:
#             currLine = line[:-1] #omits last character "\n"
#             print(currLine)
#             if currLine[-6:] == '[edit]': #if last 6 characters
#                 state = currLine[:-6]
#                 continue
#             if '(' in line:
#                 town = currLine[:currLine.index('(')-1] #keep text before the open parenthesis
#                 state_towns.append([state,town])
#             else:
#                 town = currLine
#                 state_towns.append([state,town])
#             data.append(currLine)
#     towns = pd.DataFrame(state_towns,columns = ['State','RegionName'])

    return towns

# get_list_of_university_towns()
# get_list_of_university_towns().groupby('State').size()

In [80]:
import re
import pandas as pd
import numpy as np
# list of unique states
stateStr = """
Ohio, Kentucky, American Samoa, Nevada, Wyoming
,National, Alabama, Maryland, Alaska, Utah
,Oregon, Montana, Illinois, Tennessee, District of Columbia
,Vermont, Idaho, Arkansas, Maine, Washington
,Hawaii, Wisconsin, Michigan, Indiana, New Jersey
,Arizona, Guam, Mississippi, Puerto Rico, North Carolina
,Texas, South Dakota, Northern Mariana Islands, Iowa, Missouri
,Connecticut, West Virginia, South Carolina, Louisiana, Kansas
,New York, Nebraska, Oklahoma, Florida, California
,Colorado, Pennsylvania, Delaware, New Mexico, Rhode Island
,Minnesota, Virgin Islands, New Hampshire, Massachusetts, Georgia
,North Dakota, Virginia
"""
#list of regionName entries string length
regNmLenStr = """
06,08,12,10,10,04,10,08,09,09,05,06,11,06,12,09,08,10,12,06,
06,06,08,05,09,06,05,06,10,28,06,06,09,06,08,09,10,35,09,15,
13,10,07,21,08,07,07,07,12,06,14,07,08,16,09,10,11,09,10,06,
11,05,06,09,10,12,06,06,11,07,08,13,07,11,05,06,06,07,10,08,
11,08,13,12,06,04,08,10,08,07,12,05,06,09,07,10,16,10,06,12,
08,07,06,06,06,11,14,11,07,06,06,12,08,10,11,06,10,14,04,11,
18,07,07,08,09,06,13,11,12,10,07,12,07,04,08,09,09,13,08,10,
16,09,10,08,06,08,12,07,11,09,07,09,06,12,06,09,07,10,09,10,
09,06,15,05,10,09,11,12,10,10,09,13,06,09,11,06,11,09,13,37,
06,13,06,09,49,07,11,12,09,11,11,07,12,10,06,06,09,04,09,15,
10,12,05,09,08,09,09,07,14,06,07,16,12,09,07,09,06,32,07,08,
08,06,10,36,09,10,09,06,09,11,09,06,10,07,14,08,07,06,10,09,
05,11,07,06,08,07,05,07,07,04,06,05,09,04,25,06,07,08,05,08,
06,05,11,09,07,07,06,13,09,05,16,05,10,09,08,11,06,06,06,10,
09,07,06,07,10,05,08,07,06,08,06,30,09,07,06,11,07,12,08,09,
16,12,11,08,06,04,10,10,15,05,11,11,09,08,06,04,10,10,07,09,
11,08,26,07,13,05,11,03,08,07,06,05,08,13,10,08,08,08,07,07,
09,05,04,11,11,07,06,10,11,03,04,06,06,08,08,06,10,09,05,11,
07,09,06,12,13,09,10,11,08,07,07,08,09,10,08,10,08,56,07,12,
07,16,08,04,10,10,10,10,07,09,08,09,09,10,07,09,09,09,12,14,
10,29,19,07,11,12,13,13,09,10,12,12,12,08,10,07,10,07,07,08,
08,08,09,10,09,11,09,07,09,10,11,11,10,09,09,12,09,06,08,07,
12,09,07,07,06,06,08,06,15,08,06,06,10,10,10,07,05,10,07,11,
09,12,10,12,04,10,05,05,04,14,07,10,09,07,11,10,10,10,11,15,
09,14,12,09,09,07,12,04,10,10,06,10,07,28,06,10,08,09,10,10,
10,13,12,08,10,09,09,07,09,09,07,11,11,13,08,10,07
"""

df = get_list_of_university_towns()

cols = ["State", "RegionName"]

print('Shape test: ', "Passed" if df.shape ==
      (517, 2) else 'Failed')
print('Index test: ',
      "Passed" if df.index.tolist() == list(range(517))
      else 'Failed')

print('Column test: ',
      "Passed" if df.columns.tolist() == cols else 'Failed')
print('\\n test: ',
      "Failed" if any(df[cols[0]].str.contains(
          '\n')) or any(df[cols[1]].str.contains('\n'))
      else 'Passed')
print('Trailing whitespace test:',
      "Failed" if any(df[cols[0]].str.contains(
          '\s+$')) or any(df[cols[1]].str.contains(
              '\s+$'))
      else 'Passed')
print('"(" test:',
      "Failed" if any(df[cols[0]].str.contains(
          '\(')) or any(df[cols[1]].str.contains(
              '\('))
      else 'Passed')
print('"[" test:',
      "Failed" if any(df[cols[0]].str.contains(
          '\[')) or any(df[cols[1]].str.contains(
              '\]'))
      else 'Passed')

states_vlist = [st.strip() for st in stateStr.split(',')]

mismatchedStates = df[~df['State'].isin(
    states_vlist)].loc[:, 'State'].unique()
print('State test: ', "Passed" if len(
    mismatchedStates) == 0 else "Failed")
if len(mismatchedStates) > 0:
    print()
    print('The following states failed the equality test:')
    print()
    print('\n'.join(mismatchedStates))

df['expected_length'] = [int(s.strip())
                         for s in regNmLenStr.split(',')
                         if s.strip().isdigit()]
regDiff = df[df['RegionName'].str.len() != df['expected_length']].loc[
    :, ['RegionName', 'expected_length']]
regDiff['actual_length'] = regDiff['RegionName'].str.len()
print('RegionName test: ', "Passed" if len(regDiff) ==
      0 else ' \nMismatching regionNames\n {}'.format(regDiff))

Shape test:  Passed
Index test:  Passed
Column test:  Passed
\n test:  Passed
Trailing whitespace test: Passed
"(" test: Passed
"[" test: Passed
State test:  Passed
RegionName test:  Passed


In [4]:
def read_gdp():
    
    # skipping first 4 rows
    GDP = pd.read_excel('gdplev.xls', skiprows=4) 
    # keeping columns 4 and 6
    GDP = GDP.drop(GDP.columns[[0,1,2,3,5,7]],axis=1)
    
    # assign new header
    new_header = GDP.iloc[0]
    # remove 2 blank rows
    GDP = GDP[3:]
    GDP.columns = new_header
    
    # reassign column names
    GDP = GDP.reset_index(drop=True)
    GDP.columns = ['Quarter','GDP']
    
    # only look at GDP data from the first quarter of 2000 onward
    # so drop data before 2000 Q1:
    GDP = GDP.drop(GDP.index[0:212])
   
    GDP = GDP.reset_index(drop=True)
    
    # compute difference between successive rows
    GDP['diff'] = GDP['GDP'].diff() 
    
    return GDP

In [5]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    GDP = read_gdp()

    
    # Find all the quarters where there is a decline (diff < 0)
    GDP_dec = GDP.where(GDP['diff']<0)
    GDP_dec = GDP_dec.dropna()
    
    # Find the first quarter with a successive decline
    # Get index values
    GDP_dec['Index'] = GDP_dec.index 
    # Find differences for index
    GDP_dec['Idiff'] = GDP_dec['Index'].diff() 

    # idxmin() gives the first occurence of the minimum value
    min_index = GDP_dec['Idiff'].idxmin() 

#     print(GDP_dec)
#     print(min_index)
    
    # First quarter of the 2 successive quarters with a decline 
    return GDP['Quarter'].iloc[min_index-1]

# get_recession_start()

'2008q3'

In [6]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
       
    GDP = read_gdp()
    
    # Finds all the quarters where there is a rise (diff > 0)
    GDP_rise = GDP.where(GDP['diff']>0)
    GDP_rise = GDP_rise.dropna()
    
    # Find the first quarter after the recession starts where there is a successive rise
    # Get index values
    GDP_rise['Index'] = GDP_rise.index
    # Find differences for index
    GDP_rise['Idiff'] = GDP_rise['Index'].diff()
    
    # idxmax() gives the first occurence of the maximum value
    max_index = GDP_rise['Idiff'].idxmax()
       
    # Recession ends at the second quarter of growth so max_index+1.
    return GDP['Quarter'].iloc[max_index+1]
       
# get_recession_end()

'2009q4'

In [7]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    GDP = read_gdp()
    
    # Get indices of recession start and end quarters
    start_index = GDP.loc[GDP['Quarter'] == get_recession_start()].index.astype(int)[0]
    end_index = GDP.loc[GDP['Quarter'] == get_recession_end()].index.astype(int)[0]
    
    # Just look at recession range
    GDP = GDP.iloc[start_index:end_index+1]
    
    # Return the quarter of the entry where GDP = minimum GDP
    return GDP['Quarter'][GDP.loc[GDP['GDP'] == GDP['GDP'].min()].index.astype(float)[0]]

# get_recession_bottom()

'2009q2'

In [8]:
# Change year and month column header to year and quarter
def change_to_quarter(date: str):
    date = date.split('-')
    month = int(date[1])
    quarter = int((month - 1) / 3) + 1
    return date[0] + 'q' + str(quarter)

def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    homes = pd.read_csv('City_Zhvi_AllHomes.csv')
    
    # Only looking at 2000q1 through 2016q3
    start_rem = homes.columns.get_loc('1996-04')
    end_rem = homes.columns.get_loc('2000-01')
    homes = homes.drop(homes.columns[start_rem:end_rem],axis=1) 
    
    # Keep RegionName, State and data columns
    homes = homes.drop(homes.columns[[0,3,4,5]],axis=1)
    
    # Map state names to two letter acronyms using given dictionary above
    homes['State'] = homes['State'].map(states)
    
    # Reorder State and RegionName columns
    columnsName = list(homes.columns)
    S, R = columnsName.index('State'), columnsName.index('RegionName')
    columnsName[S], columnsName[R] = columnsName[R],columnsName[S]
    homes = homes[columnsName]
    
    # Sorts and groups by index:
    homes = homes.set_index(['State','RegionName']).sort_index()
    
    # Group by quarter (Axis = 1 for column names)
    # Find mean over the months in a quarter
    homes = homes.groupby(change_to_quarter, axis=1).mean()
    
    return homes

# convert_housing_data_to_quarters()

2000q1         2000q2         2000q3  \
State   RegionName                                                 
Alabama Adamsville    69033.333333   69166.666667   69800.000000   
        Alabaster    122133.333333  123066.666667  123166.666667   
        Albertville   73966.666667   72600.000000   72833.333333   
        Arab          83766.666667   81566.666667   81333.333333   
        Ardmore                NaN            NaN            NaN   
...                            ...            ...            ...   
Wyoming Burns        101533.333333  104566.666667  108366.666667   
        Casper        89233.333333   89600.000000   89733.333333   
        Cheyenne     116866.666667  120033.333333  121533.333333   
        Evansville   128033.333333  128766.666667  130833.333333   
        Pine Bluffs   93733.333333   95066.666667   94633.333333   

                            2000q4         2001q1         2001q2  \
State   RegionName                                                 
Alabama Adamsville    71966.666667   73466.666667   74000.000000   
        Alabaster    123700.000000  123233.333333  125133.333333   
        Albertville   74200.000000   75900.000000   76000.000000   
        Arab          82966.666667   84200.000000   84533.333333   
        Ardmore                NaN            NaN            NaN   
...                            ...            ...            ...   
Wyoming Burns        113000.000000  115833.333333  117200.000000   
        Casper        93166.666667   95500.000000   97633.333333   
        Cheyenne     123633.333333  125533.333333  126300.000000   
        Evansville   132066.666667  130566.666667  131433.333333   
        Pine Bluffs   98066.666667  103233.333333  104600.000000   

                            2001q3         2001q4         2002q1  \
State   RegionName                                                 
Alabama Adamsville    73333.333333   73100.000000   73333.333333   
        Alabaster    127766.666667  127200.000000  127300.000000   
        Albertville   72066.666667   73566.666667   76533.333333   
        Arab          81666.666667   83900.000000   87266.666667   
        Ardmore                NaN            NaN            NaN   
...                            ...            ...            ...   
Wyoming Burns        117800.000000  117633.333333  117333.333333   
        Casper        99433.333333  100633.333333  101733.333333   
        Cheyenne     126466.666667  128133.333333  128466.666667   
        Evansville   132400.000000  133466.666667  133300.000000   
        Pine Bluffs  106500.000000  104066.666667  102233.333333   

                            2002q2  ...         2014q2         2014q3  \
State   RegionName                  ...                                 
Alabama Adamsville    73133.333333  ...   77066.666667   75966.666667   
        Alabaster    128000.000000  ...  147133.333333  147633.333333   
        Albertville   76366.666667  ...   84033.333333   84766.666667   
        Arab          87700.000000  ...  113366.666667  111700.000000   
        Ardmore                NaN  ...  140533.333333  139566.666667   
...                            ...  ...            ...            ...   
Wyoming Burns        117233.333333  ...  168866.666667  161933.333333   
        Casper       101533.333333  ...  175766.666667  177300.000000   
        Cheyenne     129633.333333  ...  177466.666667  176733.333333   
        Evansville   131066.666667  ...  296733.333333  305666.666667   
        Pine Bluffs  103566.666667  ...  148666.666667  154366.666667   

                            2014q4         2015q1         2015q2  \
State   RegionName                                                 
Alabama Adamsville    71900.000000   71666.666667   73033.333333   
        Alabaster    148700.000000  148900.000000  149566.666667   
        Albertville   86800.000000   88466.666667   89500.000000   
        Arab         111600.000000  110166.666667  109433.333333   
        Ardmore

In [9]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    df = convert_housing_data_to_quarters()
    
    # Start position is the quarter before the recession starts
    before_rec = (df.columns.get_loc(get_recession_start())-1)
    rec_bottom = df.columns.get_loc(get_recession_bottom())
    
    uni = get_list_of_university_towns().set_index(['State', 'RegionName'])
    
    # Turn the divided values into a DataFrame
    df = np.divide(df.iloc[:,before_rec],df.iloc[:,rec_bottom]).to_frame().dropna()
    
    # Merge university and GDP data
    uni_df = df.merge(uni, right_index=True, left_index=True, how='inner')
    
    # Drop the indices of university towns to get non-university towns
    nonuni_df = df.drop(uni_df.index)
    
    # Perform t-test
    p_value = ttest_ind(uni_df.values, nonuni_df.values).pvalue
    
    if p_value < 0.01:
        different = True
    else:
        different = False
        
    # Compare to see which has lower mean price ratio
    if uni_df.mean().values < nonuni_df.mean().values:
        better = 'university town'
    else:
        better = 'non-university town'

    return (different, p_value[0], better)
    
# run_ttest()

(True, 0.002724063704761164, 'university town')